# Create an AI-Powered Marketing Brochure with OpenAI – Featuring Multilingual Translation Capabilities


In [50]:
import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [31]:
from IPython.display import Markdown, display

In [3]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key looks good so far


# Setting up class to extract URl

In [4]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

# Getting all the url from a single website ( for use case I used Nebula)

In [5]:
nebula = Website("https://nebula.io/")
nebula.links

['/',
 '#features',
 '#benefits',
 '#about',
 'https://nebula.io/insights/',
 'https://app.nebula.io/',
 'https://calendar.app.google/uvBKvWmY3yZHdgSC8',
 '/',
 '#features',
 '#benefits',
 '#about',
 'https://nebula.io/insights/',
 'https://app.nebula.io/',
 'https://calendar.app.google/uvBKvWmY3yZHdgSC8',
 'https://calendar.app.google/uvBKvWmY3yZHdgSC8',
 'https://app.nebula.io/register',
 'https://calendar.app.google/uvBKvWmY3yZHdgSC8',
 'https://calendar.app.google/uvBKvWmY3yZHdgSC8',
 'https://calendar.app.google/uvBKvWmY3yZHdgSC8',
 'https://calendar.app.google/uvBKvWmY3yZHdgSC8',
 'https://www.linkedin.com/in/gqrgm/',
 'https://www.linkedin.com/in/eddonner/',
 'https://www.linkedin.com/in/jonkrohn/',
 'https://www.linkedin.com/in/saascfo/',
 'https://www.linkedin.com/in/mike-e-424a1819/',
 'https://www.linkedin.com/in/gqrgm/',
 'https://www.linkedin.com/in/eddonner/',
 'https://www.linkedin.com/in/jonkrohn/',
 'https://www.linkedin.com/in/saascfo/',
 'https://www.linkedin.com/in/

# Setting up prompt and tagging links

In [6]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About us, or a Company page, features, benefits, insights, request demo or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"},
        {"type": "features page": "url": "https://another.full.url/features"},
        {"type": "benefits page": "url": "https://another.full.url/benefits"},
        {"type": "insights page": "url": "https://another.full.url/insights"},
        {"type": "request demo page": "url": "https://another.full.url/request_demo"}
    ]
}
"""

In [7]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About us, or a Company page, features, benefits, insights, request demo or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"},
        {"type": "features page": "url": "https://another.full.url/features"},
        {"type": "benefits page": "url": "https://another.full.url/benefits"},
        {"type": "insights page": "url": "https://another.full.url/insights"},
        {"type": "request demo page": "url": "https://another.full.url/request_demo"}
    ]
}



In [8]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [19]:
print(get_links_user_prompt(nebula))

Here is the list of links on the website of https://nebula.io/ - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
/
#features
#benefits
#about
https://nebula.io/insights/
https://app.nebula.io/
https://calendar.app.google/uvBKvWmY3yZHdgSC8
/
#features
#benefits
#about
https://nebula.io/insights/
https://app.nebula.io/
https://calendar.app.google/uvBKvWmY3yZHdgSC8
https://calendar.app.google/uvBKvWmY3yZHdgSC8
https://app.nebula.io/register
https://calendar.app.google/uvBKvWmY3yZHdgSC8
https://calendar.app.google/uvBKvWmY3yZHdgSC8
https://calendar.app.google/uvBKvWmY3yZHdgSC8
https://calendar.app.google/uvBKvWmY3yZHdgSC8
https://www.linkedin.com/in/gqrgm/
https://www.linkedin.com/in/eddonner/
https://www.linkedin.com/in/jonkrohn/
https://www.linkedin.com/in/saascfo/
https://www.linkedin.com/in/mike-e-424a1819/
ht

# Setting up sto build company brochure

In [20]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [21]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [22]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

In [23]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [57]:
#get_brochure_user_prompt('nebula',"https://nebula.io/")

In [32]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

# Generating Brochure

In [33]:
create_brochure("nebula", "https://nebula.io/")

Found links: {'links': [{'type': 'about page', 'url': 'https://nebula.io/about'}, {'type': 'features page', 'url': 'https://nebula.io/#features'}, {'type': 'benefits page', 'url': 'https://nebula.io/#benefits'}, {'type': 'insights page', 'url': 'https://nebula.io/insights/'}, {'type': 'request demo page', 'url': 'https://app.nebula.io/register'}, {'type': 'company page', 'url': 'https://www.linkedin.com/company/nebula-io'}]}


# Nebula: Intelligent Talent Acquisition Platform

## Welcome to Nebula

Nebula is revolutionizing the recruitment landscape with its Intelligent Talent Acquisition Platform, designed to streamline and enhance the hiring process for both employers and candidates. Whether you're a company looking to find exceptional talent or a professional seeking your next opportunity, Nebula provides the tools to connect, engage, and succeed.

---

## Key Features

### 1. **Search a World-Class Talent Pool**
Discover remarkable and often-overlooked talent validated by top educators and employers. Nebula ensures you have access to the best candidates available.

### 2. **Instant Matching**
Utilize our powerful matching engine to analyze job descriptions and candidate profiles. With cutting-edge algorithms and Deep Learning Neural Networks, Nebula quickly identifies candidates best suited for your roles.

### 3. **Engagement Made Easy**
Shortlist and engage with candidates effortlessly. Organize talent profiles, signal interest, and collaborate with your team to make impactful decisions—all with just a few clicks.

### 4. **Actionable Insights**
Gain insights into compensation data, employee engagement metrics, and talent trends. Our platform helps you optimize hiring quality and speed while addressing bottlenecks in your hiring process.

---

## Benefits of Nebula

### Better Talent, Less Time
Nebula enables organizations to find 12 times as many strong candidates compared to traditional keyword-based searches. This efficiency translates into more time spent on building meaningful connections rather than sifting through resumes.

### Commitment to Diversity, Equity, and Inclusion
With our ByeBias® feature, Nebula actively promotes fair and equitable hiring practices. By anonymizing sensitive information and eliminating bias, we help create a more diverse workforce.

---

## Company Culture

At Nebula, we foster a culture of innovation and inclusivity. Our team, comprised of experts in technology and recruitment, is dedicated to developing solutions that prioritize human connections and ethical practices in hiring. We believe in the power of diverse perspectives and aim to reflect that in our work environment.

---

## Leadership Team

- **Steven Talbot** - Co-Founder and CEO
- **Ed Donner** - Co-Founder and CTO
- **Jon Krohn** - Co-Founder and Chief Data Scientist
- **Alex Gardner** - CFO
- **Mike Evers** - Strategy & GTM

With a leadership team comprised of visionaries and innovators, Nebula is positioned to redefine the recruitment landscape.

---

## Careers at Nebula

Join our mission to revolutionize talent acquisition! We are always looking for passionate individuals who share our values of innovation, diversity, and excellence. Explore career opportunities with us and be part of a team that is making a difference in the recruitment industry.

---

## Request a Demo

Experience the power of Nebula firsthand! [Request a demo](#) or [try it for free](#) to see how we can transform your hiring process.

Explore more about what we offer and how we can support your talent acquisition needs at [Nebula.io](https://nebula.io).

---

### Connect With Us

For more insights and updates, follow us on our social media channels or contact us for further information. 

**© 2023 Nebula.io, LLC. All Rights Reserved.**  
**Terms of Service | Privacy Policy**  
**Nebula.io’s use of information adheres to Google API Services User Data Policy.**

# Streaming Brochure

In [34]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [35]:
stream_brochure("nebula", "https://nebula.io/")

Found links: {'links': [{'type': 'about page', 'url': 'https://nebula.io/#about'}, {'type': 'features page', 'url': 'https://nebula.io/#features'}, {'type': 'benefits page', 'url': 'https://nebula.io/#benefits'}, {'type': 'insights page', 'url': 'https://nebula.io/insights/'}, {'type': 'request demo page', 'url': 'https://app.nebula.io/register'}, {'type': 'company page', 'url': 'https://www.linkedin.com/company/nebula-io'}]}


# Nebula Brochure

## Introducing Nebula
Welcome to **Nebula**, the Intelligent Talent Acquisition Platform that reimagines the way recruitment is conducted. Our platform empowers organizations to efficiently discover and engage the world's best talent, transforming traditional hiring practices into a streamlined, data-driven process.

---

## **Features**
### Unleash the Power of Talent Acquisition
1. **Search**  
   Explore a vast pool of talent validated by top sources, educators, and employers. Uncover the exceptional candidates often overlooked by traditional job boards.
   
2. **Instant Matching**  
   Enter a job description or resume, and let our algorithm process millions of profiles in seconds. Our unique deep learning neural networks ensure you find the best candidates swiftly.

3. **Engagement**  
   Shortlist candidates with ease and organize profiles for current or future roles. Collaborate with your hiring team seamlessly to enhance decision-making.

4. **Actionable Insights**  
   Gain real-time insights into compensation data, employee engagement, and industry talent trends. Optimize hiring quality and efficiency to eliminate bottlenecks.

---

## **Our Mission**
At Nebula, we believe in improving recruitment processes while being human-centric. We provide the right tools to help organizations connect with candidates more effectively, fostering rich human interactions in hiring.

---

## **Benefits**
### Better Talent, Less Time
An independent assessment from a Fortune 500 client revealed that Nebula’s innovative approach identifies 12x more strong candidates faster and enhances the time spent engaging with candidates, allowing for more genuine connections.

### **Diversity, Equity & Inclusion**
Fair hiring practices are at our core. Our **ByeBias®** feature ensures that bias, both conscious and unconscious, is mitigated throughout the recruitment process. We enable you to champion diversity in your hiring initiatives.

---

## **Company Culture**
At Nebula, we thrive on innovation, collaboration, and inclusivity. Our team consists of forward-thinkers dedicated to building a platform that empowers organizations and candidates alike. We promote a strong culture of transparency and support, working tirelessly to foster an environment where every voice is heard.

## **Leadership Team**
- **Steven Talbot** - Co-Founder and CEO
- **Ed Donner** - Co-Founder and CTO
- **Jon Krohn** - Co-Founder and Chief Data Scientist
- **Alex Gardner** - CFO
- **Mike Evers** - Strategy & GTM

---

## **Careers at Nebula**
Join our dynamic team at Nebula! We are always looking for passionate individuals who are committed to transforming the recruitment landscape. Explore exciting career opportunities where you can make a real impact and grow professionally. Find out more on our **Careers** page.

---

## **Get Started Today**
Ready to revolutionize your recruitment process?  
- Schedule a demo or try us for free at [Nebula.io](https://www.nebula.io)  
- Contact our sales team for more information.

--- 

> **Contact Us**: Reach us at [support@nebula.io](mailto:support@nebula.io)  
> **Follow us**: Stay updated on LinkedIn and other social platforms.

# Setting up for Language Translation

In [54]:
from IPython.display import clear_output, display, Markdown

def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
    
    return response  # Return response without displaying it

def translate_text(text, target_language):
    translation = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": "Translate the following text into " + target_language},
            {"role": "user", "content": text}
        ]
    )
    return translation.choices[0].message.content

def user_translate_brochure(company_name, url, lang):
    clear_output(wait=True)
    brochure_text = stream_brochure(company_name, url)  # Fetch text without displaying
    translated_text = translate_text(brochure_text, lang)  # Translate it
    display(Markdown(translated_text))  # Display only the translated text


# Example of French Translation

In [55]:
company_name = "nebula"
url = "https://nebula.io/"

def user_translate_brochure(company_name, url, lang):
    # Clear previous output
    clear_output(wait=True)
    
    # Get original brochure text
    brochure_text = stream_brochure(company_name, url)

    # Stream translation
    translation_stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": f"You are a translator. Convert the following text into {lang} without asking for further input."},
            {"role": "user", "content": brochure_text}
        ],
        stream=True
    )
    
    # Setup display for streaming translation
    display_handle = display(Markdown(""), display_id=True)
    translated_text = ""
    
    for chunk in translation_stream:
        content = chunk.choices[0].delta.content or ""
        if content:
            translated_text += content
            update_display(Markdown(translated_text), display_id=display_handle.display_id)

# Stream the brochure in English
original_text = stream_brochure(company_name, url)

# Prompt user for language choice
language_choice = input(" Enter language to translate brochure (e.g., 'Greek'): ")

# Translate the brochure and stream the translation
if language_choice:
    user_translate_brochure(company_name, url, language_choice)


Found links: {'links': [{'type': 'about page', 'url': 'https://nebula.io/about'}, {'type': 'features page', 'url': 'https://nebula.io#features'}, {'type': 'benefits page', 'url': 'https://nebula.io#benefits'}, {'type': 'insights page', 'url': 'https://nebula.io/insights/'}, {'type': 'request demo page', 'url': 'https://app.nebula.io/register'}, {'type': 'company page', 'url': 'https://www.linkedin.com/company/nebula-io'}]}


# Brochure de l'entreprise Nebula

## Bienvenue chez Nebula
**Réinventer l'acquisition de talents avec intelligence**

Chez Nebula, nous croyons que trouver et acquérir les meilleurs talents ne devrait pas seulement être plus efficace, mais aussi plus humain. Notre plateforme d'acquisition de talents intelligente utilise une technologie AI avancée pour débloquer l'accès aux professionnels les plus talentueux du monde, offrant des expériences de recrutement plus intelligentes et rationalisées.

---

## Caractéristiques de notre plateforme

1. **Réservoir de talents de classe mondiale**  
   Découvrez des talents exceptionnels et cachés qui passent souvent inaperçus sur les sites d'emploi traditionnels.
   
2. **Correspondance instantanée**  
   Utilisez nos puissants algorithmes pour évaluer, trier et classer les profils en fonction de vos besoins en recrutement, trouvant les meilleures correspondances en quelques secondes.

3. **Engagement facile**  
   Organisez les profils des candidats sans effort et signalez votre intérêt d'un simple clic. Collaborez sans friction avec votre équipe pour prendre des décisions d'embauche impactantes.

4. **Insights exploitables**  
   Obtenez des informations précieuses sur les tendances en matière de talents, les données de rémunération et l'engagement des employés pour renforcer votre planification des ressources humaines et vos stratégies de fidélisation.

---

## Avantages de choisir Nebula

- **Meilleurs talents, plus rapidement** : Des évaluations indépendantes montrent que Nebula identifie jusqu'à 12 fois plus de candidats forts en beaucoup moins de temps, améliorant la qualité des recrues.
- **Mise l'accent sur l'humanité** : Avec moins de temps passé sur des recherches laborieuses, les équipes peuvent privilégier les interactions significatives, favorisant les connexions humaines.
- **Diversité, équité et inclusion** : Grâce à nos fonctionnalités ByeBias®, nous promouvons un recrutement équitable en éliminant les biais du processus de recrutement, soutenant une main-d'œuvre inclusive.

---

## Notre équipe de direction

**Steven Talbot** – Co-fondateur et PDG  
**Ed Donner** – Co-fondateur et CTO  
**Jon Krohn** – Co-fondateur et Chief Data Scientist  
**Alex Gardner** – CFO  
**Mike Evers** – Stratégie & GTM  

---

## Culture d'entreprise

Chez Nebula, nous favorisons un environnement de collaboration et d'innovation où les idées de chaque membre de l'équipe sont valorisées. Nous nous engageons en faveur de l'inclusivité, en défendant la diversité à tous les niveaux de nos opérations. Notre culture met l'accent sur l'apprentissage continu, la croissance professionnelle et un engagement à atteindre des objectifs communs.

---

## Rejoignez notre équipe !

Nous sommes toujours à la recherche de personnes talentueuses passionnées par la transformation du paysage du recrutement. Si vous souhaitez faire partie d'une équipe dédiée à l'innovation, à la connexion humaine et à l'utilisation de la technologie pour le bien social, envisagez une carrière chez Nebula.

**Explorez les opportunités de carrière :** [Rejoignez-nous](#)

---

## Clients
Nebula sert une gamme diversifiée de clients, y compris des entreprises du Fortune 500, des petites entreprises et des établissements d'enseignement cherchant à optimiser leurs processus de recrutement. 

---

## Connectez-vous avec nous

Êtes-vous prêt à transformer votre processus de recrutement ?  
**[Planifiez une démo](#)** | **[Essayez-le gratuitement](#)**

*Clause de non-responsabilité : Tous droits réservés. Les conditions de service et la politique de confidentialité s'appliquent.*

--- 

Adoptez l'avenir du recrutement avec Nebula, où des solutions intelligentes rencontrent un recrutement centré sur l'humain.

# Example of Spanish  Translation

In [56]:

original_text = stream_brochure(company_name, url)

# Prompt user for language choice
language_choice = input(" Enter language to translate brochure (e.g., 'Greek'): ")

# Translate the brochure and stream the translation
if language_choice:
    user_translate_brochure(company_name, url, language_choice)


Found links: {'links': [{'type': 'about page', 'url': 'https://nebula.io/#about'}, {'type': 'features page', 'url': 'https://nebula.io/#features'}, {'type': 'benefits page', 'url': 'https://nebula.io/#benefits'}, {'type': 'insights page', 'url': 'https://nebula.io/insights/'}, {'type': 'request demo page', 'url': 'https://app.nebula.io/register'}, {'type': 'company page', 'url': 'https://nebula.io/'}]}


# Nebula: Reimaginando el Reclutamiento con Adquisición de Talentos Inteligente

## Acerca de nosotros
Nebula es una Plataforma de Adquisición de Talentos Inteligente que revoluciona los procesos de reclutamiento. Nuestra misión es desbloquear el mejor talento del mundo mediante la utilización de tecnología innovadora, asegurando que la búsqueda de talento se vuelva más eficiente, efectiva y equitativa. Nuestra plataforma está diseñada para organizaciones de todos los tamaños que buscan mejorar sus estrategias de contratación y descubrir candidatos excepcionales.

## Características
### 1. Búsqueda
Accede a una base de datos de talentos de clase mundial, validada por fuentes líderes, educadores y empleadores. Encuentra talento excepcional que a menudo es pasado por alto por las bolsas de trabajo tradicionales.

### 2. Coincidencia Instantánea
Utiliza algoritmos avanzados que procesan Lenguaje Natural con Redes Neuronales de Aprendizaje Profundo. Nuestro motor de coincidencia clasifica, ordena y prioriza perfiles rápidamente según las descripciones de trabajo proporcionadas o arquetipos de candidatos, analizando cientos de millones de perfiles en cuestión de segundos.

### 3. Compromiso
Organiza y comprométete con el talento interesado sin esfuerzo. Agrega perfiles rápidamente, programa reuniones y colabora sin problemas con miembros del equipo y gerentes de contratación.

### 4. Perspectivas Accionables
Obtén valiosas perspectivas sobre la planificación, adquisición y retención de talento. Nuestra plataforma proporciona datos de compensación en tiempo real, métricas de compromiso de empleados e información sobre las tendencias actuales de talento para mejorar tu eficiencia de contratación.

## Beneficios
### Más Candidatos, Menos Tiempo
El enfoque innovador de Nebula permite la identificación de **12x más candidatos fuertes** en comparación con las búsquedas tradicionales basadas en palabras clave, reduciendo significativamente el tiempo dedicado a la búsqueda y aumentando el tiempo dedicado a construir conexiones humanas significativas.

### Diversidad, Equidad e Inclusión
Con nuestra función **ByeBias®**, Nebula anonimiza y agrega información sensible para mitigar los sesgos conscientes e inconscientes durante el reclutamiento. Esto asegura prácticas de empleo justas, promoviendo la diversidad y la inclusión en el proceso de contratación.

## Equipo de Liderazgo
- **Steven Talbot**: Co-Fundador y CEO
- **Ed Donner**: Co-Fundador y CTO
- **Jon Krohn**: Co-Fundador y Científico de Datos Principal
- **Alex Gardner**: CFO
- **Mike Evers**: Estrategia y GTM

## Cultura Empresarial
En Nebula, creemos en fomentar una cultura de innovación, colaboración e inclusión. Nuestro equipo se dedica a crear un entorno de trabajo diverso y de apoyo donde cada empleado esté empoderado para contribuir a su máximo potencial. Valoramos el aprendizaje continuo y la adaptabilidad, nutriendo el talento no solo para tener éxito, sino para prosperar.

## Carreras en Nebula
¡Únete a nosotros en la redefinición del futuro del reclutamiento! Estamos continuamente buscando personas apasionadas para ayudarnos a desarrollar aún más nuestra plataforma. Si estás interesado en trabajar con tecnología de IA de vanguardia y hacer un impacto duradero en el mundo de la tecnología de recursos humanos, explora nuestros puestos disponibles y aplica hoy.

## Conéctate con nosotros
Descubre el futuro del reclutamiento con Nebula. **[Solicita una Demo](#)** o **[Pruébalo Gratis](#)** hoy mismo!

---

© 2023 Nebula.io, LLC. Todos los derechos reservados.